In [2]:
import os 
import json
import re
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np

RUNS = [
    {
        "name": "FasterRCNN - Original",
        "gt":   r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\Coco_output\Coco_output\exdark_test.json",
        "dt":   r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs\exdark_frcnn_outputs\test_result\coco_test_detections.json",
        "out_dir": r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs\exdark_frcnn_outputs\test_result\overall_test_result.json"
    },
    {
        "name": "FasterRCNN - ZeroDCE++",
        "gt":   r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\Coco_output\Coco_output\exdark_test.json",
        "dt":   r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_zero-dce\exdark_frcnn_outputs_zero-dce\test_result\coco_test_detections.json",
        "out_dir": r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_zero-dce\exdark_frcnn_outputs_zero-dce\test_result\overall_test_result.json"
    },
    {
        "name": "FasterRCNN - ZeroDCE++ + NAFNet",
        "gt":   r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\resized_coco_json\resized_coco_json\instances_test.json",
        "dt":  r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_nafnet\exdark_frcnn_outputs_nafnet\test_result\coco_test_detections.json",
        "out_dir": r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_nafnet\exdark_frcnn_outputs_nafnet\test_result\overall_test_result.json"
    }]


def slug(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9]+", "_", s).strip("_").lower()

def eval_one(gt_path, dt_path):
    # Load GT and patch optional headers some JSONs omit
    cocoGt = COCO(gt_path)
    cocoGt.dataset.setdefault("info", {})
    cocoGt.dataset.setdefault("licenses", [])

    # Load detections (COCO list of dicts)
    preds = json.load(open(dt_path, "r"))

    # Defensive filter: keep only detections on GT image/category ids
    gt_img_ids = {im["id"] for im in cocoGt.dataset["images"]}
    gt_cat_ids = {c["id"] for c in cocoGt.dataset["categories"]}
    preds = [d for d in preds if d.get("image_id") in gt_img_ids and d.get("category_id") in gt_cat_ids]
    if not preds:
        raise RuntimeError(f"No valid detections after filtering for {dt_path}")
    cocoDt = cocoGt.loadRes(preds)

    # Overall COCO stats (for mAP50-95 and mAP50)
    ev = COCOeval(cocoGt, cocoDt, "bbox")
    ev.evaluate(); ev.accumulate(); ev.summarize()
    stats = ev.stats  # [AP, AP50, AP75, APs, APm, APl, AR1, AR10, AR100, ARs, ARm, ARl]
    mAP_50_95 = float(stats[0])
    mAP_50    = float(stats[1])

    # Precision/Recall at IoU=0.5 WITHOUT using F1
    ev2 = COCOeval(cocoGt, cocoDt, "bbox")
    ev2.params.iouThrs = np.array([0.5], dtype=np.float64)  # single PR curve at 0.5
    ev2.evaluate(); ev2.accumulate()

    # precision shape: [T, R, K, A, M]; recall shape: [T, K, A, M]
    p = ev2.eval["precision"]; r_grid = ev2.params.recThrs
    aind, mind, t0 = 0, -1, 0

    # class-averaged precision across recall grid
    p_mean = np.where(p[t0, :, :, aind, mind] < 0, np.nan, p[t0, :, :, aind, mind]).mean(axis=1)  # [R]

    # R@0.5 = average maximum recall across classes (IoU=0.5)
    rec = ev2.eval["recall"][t0, :, aind, mind]  # [K]
    R_50 = float(np.nanmean(np.where(rec < 0, np.nan, rec)))

    # P@0.5 reported at recall ≈ 0.50 (no F1 dependence)
    idx50 = int(np.argmin(np.abs(r_grid - 0.50)))
    P_50 = float(p_mean[idx50]) if not np.isnan(p_mean[idx50]) else float("nan")

    return {
        "images": len(cocoGt.imgs),
        "instances": len(cocoGt.getAnnIds()),
        "P@0.5": P_50, "R@0.5": R_50,
        "mAP50": mAP_50, "mAP50-95": mAP_50_95
    }

def main():
    print("\nClass     Images  Instances      Box(P    R   mAP50  mAP50-95)")
    for run in RUNS:
        name, gt, dt, out_dir = run["name"], run["gt"], run["dt"], run["out_dir"]
        os.makedirs(out_dir, exist_ok=True)
        m = eval_one(gt, dt)

        # Console line (YOLOv8-style)
        print(f"all  ({name}) {m['images']:7d}  {m['instances']:10d}      {m['P@0.5']:.3f} {m['R@0.5']:.3f}  {m['mAP50']:.3f}    {m['mAP50-95']:.3f}")

        # Per-run CSV
        out_csv = os.path.join(out_dir, f"overall_summary_{slug(name)}.csv")
        with open(out_csv, "w") as f:
            f.write("Class,Images,Instances,P@0.5,R@0.5,mAP50,mAP50-95\n")
            f.write(f"all,{m['images']},{m['instances']},{m['P@0.5']:.4f},{m['R@0.5']:.4f},{m['mAP50']:.4f},{m['mAP50-95']:.4f}\n")
        print("Saved:", out_csv)

if __name__ == "__main__":
    main()


Class     Images  Instances      Box(P    R   mAP50  mAP50-95)
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.75s).
Accumulating evaluation results...
DONE (t=0.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.493
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.294
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.241
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.394
 

In [2]:
pip install pycocotools


   ---------------------------------------- 0.0/76.8 kB ? eta -:--:--
   ---------- ----------------------------- 20.5/76.8 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 76.8/76.8 kB 853.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Faster R-CNN

In [ ]:
import pandas as pd

df_original = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs\exdark_frcnn_outputs\test_result\lighting_stats_test.csv")
df_zero_dce = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_zero-dce\exdark_frcnn_outputs_zero-dce\test_result\lighting_stats_test.csv")
df_nafnet = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_nafnet\exdark_frcnn_outputs_nafnet\test_result\lighting_stats_test.csv")   

In [3]:
df_original


,Lighting,AP,AP50,AP75,AR@1,AR@10,AR@100,num_images
0,Low,0.2918,0.5203,0.2590,0.2692,0.3924,0.3937,134
1,Ambient,0.2361,0.4678,0.2177,0.2536,0.3785,0.3899,925
2,Object,0.2494,0.5016,0.2164,0.2364,0.3700,0.3749,144
3,Single,0.2769,0.5303,0.2600,0.2715,0.4253,0.4293,255
4,Weak,0.2536,0.4886,0.2475,0.2187,0.3740,0.3796,232
5,Strong,0.2880,0.5355,0.2833,0.2380,0.3945,0.4001,538
6,Screen,0.3093,0.6377,0.1991,0.2018,0.4252,0.4294,28
7,Window,0.2664,0.5274,0.2341,0.2577,0.4069,0.4116,119
8,Shadow,0.3175,0.6412,0.2515,0.2919,0.3794,0.3799,14
9,Twilight,0.3470,0.5902,0.3582,0.3586,0.4607,0.4615,174


In [4]:
df_nafnet

,Lighting,AP,AP50,AP75,AR@1,AR@10,AR@100,num_images
0,Low,0.2846,0.4757,0.2848,0.2911,0.3887,0.3892,134
1,Ambient,0.2265,0.4520,0.2097,0.2488,0.3746,0.3831,925
2,Object,0.2225,0.4323,0.1877,0.2216,0.3709,0.3744,144
3,Single,0.2587,0.4677,0.2649,0.2575,0.3683,0.3757,255
4,Weak,0.2385,0.4827,0.2164,0.2076,0.3561,0.3636,232
5,Strong,0.2700,0.5079,0.2460,0.2303,0.3747,0.3777,538
6,Screen,0.3689,0.6403,0.4420,0.2379,0.4928,0.4928,28
7,Window,0.2355,0.4936,0.1914,0.2353,0.3622,0.3681,119
8,Shadow,0.3722,0.8275,0.2122,0.3118,0.3911,0.3951,14
9,Twilight,0.3851,0.6506,0.4667,0.3500,0.4616,0.4629,174


In [5]:
df_zero_dce

,Lighting,AP,AP50,AP75,AR@1,AR@10,AR@100,num_images
0,Low,0.2629,0.4386,0.3084,0.2743,0.3488,0.3720,134
1,Ambient,0.2250,0.4553,0.1995,0.2448,0.3706,0.3834,925
2,Object,0.2234,0.4497,0.1830,0.2187,0.3384,0.3434,144
3,Single,0.2688,0.5060,0.2662,0.2632,0.3915,0.3951,255
4,Weak,0.2473,0.4679,0.2540,0.2049,0.3641,0.3698,232
5,Strong,0.2736,0.5070,0.2779,0.2300,0.3795,0.3827,538
6,Screen,0.3397,0.6238,0.2033,0.2263,0.4400,0.4400,28
7,Window,0.2487,0.5149,0.1867,0.2592,0.3895,0.3941,119
8,Shadow,0.3830,0.8588,0.2696,0.3259,0.4092,0.4105,14
9,Twilight,0.3302,0.5954,0.3394,0.3188,0.4193,0.4229,174


### Faster R-CNN

per - lighting condition

In [25]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({
    "font.size": 16,         
    "axes.titlesize": 20,   
    "axes.labelsize": 18,   
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 14,
    "legend.title_fontsize": 16
})

df_original = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs\exdark_frcnn_outputs\test_result\lighting_stats_test.csv")
df_zero_dce = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_zero-dce\exdark_frcnn_outputs_zero-dce\test_result\lighting_stats_test.csv")
df_nafnet   = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_nafnet\exdark_frcnn_outputs_nafnet\test_result\lighting_stats_test.csv")   

out_dir = r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result_lighting_stats"
os.makedirs(out_dir, exist_ok=True)
LIGHTING_ORDER = ["Low","Ambient","Object","Single","Weak","Strong","Screen","Window","Shadow","Twilight"]

def load_tag(df, label):
    df = df.copy()
    cols = [c for c in ["Lighting","AP","AP50","AP75","AR@1","AR@10","AR@100","num_images"] if c in df.columns]
    df = df[cols]
    for c in df.columns:
        if c != "Lighting":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df["Model"] = label
    return df

df = pd.concat([
    load_tag(df_original, "Original + Faster R-CNN"),
    load_tag(df_zero_dce, "Zero-DCE++ + Faster R-CNN"),
    load_tag(df_nafnet,   "Zero-DCE++ + NAFNet + Faster R-CNN")
], ignore_index=True)

present = df["Lighting"].unique().tolist()
order = [l for l in LIGHTING_ORDER if l in present]
df["Lighting"] = pd.Categorical(df["Lighting"], categories=order, ordered=True)

COLOR = {
    "Original + Faster R-CNN": "#224ede",
    "Zero-DCE++ + Faster R-CNN": "#45d867",
    "Zero-DCE++ + NAFNet + Faster R-CNN": "#f88622",
}

def plot_grouped(metric, title, ylabel, fname, ylim=None):
    if metric not in df.columns:
        print(f"Skip {metric}: not in CSV"); return

    pivot = df.pivot_table(index="Lighting", columns="Model",
                           values=metric, aggfunc="mean").loc[order]
    models = list(pivot.columns)
    x = np.arange(len(pivot.index))
    w = 0.9 / max(1, len(models))

    fig, ax = plt.subplots(figsize=(16, 10))
    for i, m in enumerate(models):
        ax.bar(x + (i - (len(models)-1)/2)*w, pivot[m].values, width=w,
               label=m, color=COLOR.get(m, None))
        for xi, yi in zip(x, pivot[m].values):
            if np.isfinite(yi):
                ax.text(xi + (i - (len(models)-1)/2)*w, yi, f"{yi:.3f}",
                        ha="center", va="bottom", fontsize=11)
    ax.set_xticks(x)
    ax.set_xticklabels(pivot.index, rotation=30, ha="right")
    ax.set_title(title, pad=12)
    ax.set_xlabel("Lighting condition", labelpad=8)
    if ylim: ax.set_ylim(ylim)
    ax.yaxis.set_visible(False)
    ax.set_ylabel("")
    for s in ["left", "right", "top"]:
        ax.spines[s].set_visible(False)
    ax.grid(False)

    ax.legend(title="Model")
    plt.tight_layout()
    out_path = os.path.join(out_dir, fname)
    plt.savefig(out_path, dpi=300)
    plt.close()
    print("Saved:", out_path)

plot_grouped("AP",     "AP (mAP50–95) by lighting", "mAP(50–95)", "AP_mAP50_95_by_lighting.png")
plot_grouped("AR@100", "Recall by lighting (AR@100)", "Recall",    "Recall_AR100_by_lighting.png")


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1783366062.py:53: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Lighting", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result_lighting_stats\AP_mAP50_95_by_lighting.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1783366062.py:53: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Lighting", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result_lighting_stats\Recall_AR100_by_lighting.png


per-class results

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({
    "font.size": 16,        
    "axes.titlesize": 22,   
    "axes.labelsize": 18,   
    "xtick.labelsize": 18,
    "ytick.labelsize": 16,
    "legend.fontsize": 15,
    "legend.title_fontsize": 18
})

df_original = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs\exdark_frcnn_outputs\test_result\per_class_ap_test.csv")
df_zero_dce = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_zero-dce\exdark_frcnn_outputs_zero-dce\test_result\per_class_ap_test.csv")
df_nafnet   = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\exdark_frcnn_outputs_nafnet\exdark_frcnn_outputs_nafnet\test_result\per_class_ap_test.csv")

out_dir = r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result"
os.makedirs(out_dir, exist_ok=True)
CLASS_ORDER = ["Bicycle","Boat","Bottle","Bus","Car","Cat","Chair","Cup","Dog","Motorbike","People","Table"]

COLOR = {
    "Original + Faster R-CNN": "#224ede",
    "Zero-DCE++ + Faster R-CNN": "#45d867",
    "Zero-DCE++ + NAFNet + Faster R-CNN": "#f88622",
}

def tag_df(df, label):
    d = df.copy()
    d.columns = [c.strip() for c in d.columns]
    if "Class" not in d.columns and "class" in d.columns:
        d = d.rename(columns={"class": "Class"})
    keep = [c for c in ["Class","AP","AP50","AP75","meanPrecision","meanRecall"] if c in d.columns]
    d = d[keep]
    for c in keep:
        if c != "Class":
            d[c] = pd.to_numeric(d[c], errors="coerce")
    d["Model"] = label
    return d

df = pd.concat([
    tag_df(df_original, "Original + Faster R-CNN"),
    tag_df(df_zero_dce, "Zero-DCE++ + Faster R-CNN"),
    tag_df(df_nafnet,   "Zero-DCE++ + NAFNet + Faster R-CNN"),
], ignore_index=True)

present = [c for c in CLASS_ORDER if c in df["Class"].unique().tolist()]
if present:
    df["Class"] = pd.Categorical(df["Class"], categories=present, ordered=True)

def plot_grouped(metric, title, fname, ylim=None):
    if metric not in df.columns:
        print(f"Skip {metric}: not in CSV"); return
    classes_present = df["Class"].dropna().unique().tolist()
    order_cls = [c for c in CLASS_ORDER if c in classes_present] or sorted(classes_present)

    pivot = df.pivot_table(index="Class", columns="Model",
                           values=metric, aggfunc="mean").loc[order_cls]

    models = list(pivot.columns)
    x = np.arange(len(pivot.index))*1.6
    w = 1.2 / max(1, len(models))  
    fig, ax = plt.subplots(figsize=(24, 14))  
    for i, m in enumerate(models):
        ax.bar(x + (i - (len(models)-1)/2)*w, pivot[m].values, width=w,
               label=m, color=COLOR.get(m, None))
        for xi, yi in zip(x, pivot[m].values):
            if np.isfinite(yi):
                ax.text(xi + (i - (len(models)-1)/2)*w, yi, f"{yi:.3f}",
                        ha="center", va="bottom", fontsize=11, fontweight="bold")  

    ax.set_xticks(x)
    ax.set_xticklabels(pivot.index, rotation=30, ha="right", fontsize=18)
    ax.set_title(title, pad=20) 
    ax.set_xlabel("Class", fontsize=20, labelpad=10)
    if ylim: ax.set_ylim(ylim)
    ax.yaxis.set_visible(False)
    ax.set_ylabel("")
    for s in ["left", "right", "top"]:
        ax.spines[s].set_visible(False)
    ax.grid(False)

    ax.legend(title="Model", fontsize=18, title_fontsize=18)
    plt.tight_layout()
    out_path = os.path.join(out_dir, fname)
    plt.savefig(out_path, dpi=300) 
    plt.close()
    print("Saved:", out_path)

plot_grouped("AP",   "Per-class AP (mAP50–95)", "per_class_AP.png")
plot_grouped("AP50", "Per-class AP@0.50",       "per_class_AP50.png")
plot_grouped("meanPrecision", "Per-class Precision (avg)", "per_class_precision.png")
plot_grouped("meanRecall",    "Per-class Recall (avg)",    "per_class_recall.png")


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1517613641.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result\per_class_AP.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1517613641.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result\per_class_AP50.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1517613641.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result\per_class_precision.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\1517613641.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\1. Faster R-CNN\test_result\per_class_recall.png


### YOLOv8

per - lighting condition

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({
    "font.size": 16,        
    "axes.titlesize": 20,  
    "axes.labelsize": 18,  
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 14,
    "legend.title_fontsize": 16
})

df_nafnet = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\nafnet\runs\runs\metrics_out\per_lighting_overall.csv")
df_original = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\original\runs\runs\metrics_out_original\per_lighting_overall.csv")
df_zerodce = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\zerodce\runs\runs\metrics_out_zerodce\per_lighting_overall.csv")

out_dir = r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result"
os.makedirs(out_dir, exist_ok=True)

LIGHTING_ORDER = ["Low","Ambient","Object","Single","Weak","Strong","Screen","Window","Shadow","Twilight"]

def load_tag(df, label):
    d = df.copy()
    d.columns = [c.strip() for c in d.columns]
    if "Lighting" not in d.columns:
        if "condition" in d.columns:
            d = d.rename(columns={"condition": "Lighting"})
        elif "Condition" in d.columns:
            d = d.rename(columns={"Condition": "Lighting"})
    if "count_images" in d.columns and "num_images" not in d.columns:
        d = d.rename(columns={"count_images": "num_images"})
    cols = [c for c in ["Lighting","count_images","AP","AP50","AP75","APs","APm","APl","AR1","AR10","AR100","ARs","ARm","ARl"] if c in d.columns]
    d = d[cols]
    for c in d.columns:
        if c != "Lighting":
            d[c] = pd.to_numeric(d[c], errors="coerce")
    d["Model"] = label
    return d
df = pd.concat([
    load_tag(df_original, "Original + YOLOv8"),
    load_tag(df_zerodce, "Zero-DCE++ + YOLOv8"),
    load_tag(df_nafnet,  "Zero-DCE++ + NAFNet + YOLOv8"),
], ignore_index=True)
present = df["Lighting"].unique().tolist()
order = [l for l in LIGHTING_ORDER if l in present]
df["Lighting"] = pd.Categorical(df["Lighting"], categories=order, ordered=True)

COLOR = {
    "Original + YOLOv8": "#de223b",
    "Zero-DCE++ + YOLOv8": "#74d6e3",
    "Zero-DCE++ + NAFNet + YOLOv8": "#99b753",
}

def plot_grouped(metric, title, ylabel, fname, ylim=None):
    if metric not in df.columns:
        print(f"Skip {metric}: not in CSV"); return

    pivot = df.pivot_table(index="Lighting", columns="Model",
                           values=metric, aggfunc="mean").loc[order]
    models = list(pivot.columns)
    x = np.arange(len(pivot.index))
    w = 0.8 / max(1, len(models))

    fig, ax = plt.subplots(figsize=(18, 9))
    for i, m in enumerate(models):
        ax.bar(x + (i - (len(models)-1)/2)*w, pivot[m].values, width=w,
               label=m, color=COLOR.get(m, None))
        for xi, yi in zip(x, pivot[m].values):
            if np.isfinite(yi):
                ax.text(xi + (i - (len(models)-1)/2)*w, yi, f"{yi:.3f}",
                        ha="center", va="bottom", fontsize=11)

    ax.set_xticks(x)
    ax.set_xticklabels(pivot.index, rotation=30, ha="right")
    ax.set_title(title, pad=12)
    ax.set_xlabel("Lighting condition", labelpad=8)
    if ylim: ax.set_ylim(ylim)
    ax.yaxis.set_visible(False)
    ax.set_ylabel("")
    for s in ["left", "right", "top"]:
        ax.spines[s].set_visible(False)
    ax.grid(False)

    ax.legend(title="Model")
    plt.tight_layout()
    out_path = os.path.join(out_dir, fname)
    plt.savefig(out_path, dpi=300)
    plt.close()
    print("Saved:", out_path)

plot_grouped("AP",     "AP (mAP50–95) by lighting", "mAP(50–95)", "AP_mAP50_95_by_lighting.png")
plot_grouped("AR100", "Recall by lighting (AR100)", "Recall",    "Recall_AR100_by_lighting.png")


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\3103678965.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Lighting", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result\AP_mAP50_95_by_lighting.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\3103678965.py:68: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Lighting", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result\Recall_AR100_by_lighting.png


Per - class results 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({
    "font.size": 16,       
    "axes.titlesize": 22,   
    "axes.labelsize": 18,   
    "xtick.labelsize": 18,
    "ytick.labelsize": 14,
    "legend.fontsize": 14,
    "legend.title_fontsize": 16
})

df_original = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\nafnet\runs\runs\metrics_out\per_class_metrics.csv")
df_zero_dce = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\original\runs\runs\metrics_out_original\per_class_metrics.csv")
df_nafnet   = pd.read_csv(r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\zerodce\runs\runs\metrics_out_zerodce\per_class_metrics.csv")

out_dir = r"D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result\class"
os.makedirs(out_dir, exist_ok=True)
CLASS_ORDER = ["Bicycle","Boat","Bottle","Bus","Car","Cat","Chair","Cup","Dog","Motorbike","People","Table"]

COLOR = {
    "Original + YOLOv8": "#6077c4",
    "Zero-DCE++ + YOLOv8": "#b167b4",
    "Zero-DCE++ + NAFNet + YOLOv8": "#c6ce66",
}

def tag_df(df, label):
    d = df.copy()
    d.columns = [c.strip() for c in d.columns]
    if "Class" not in d.columns:
        if "class_name" in d.columns:
            d = d.rename(columns={"class_name": "Class"})
        elif "class" in d.columns:
            d = d.rename(columns={"class": "Class"})
    keep = [c for c in ["Class","class_id","AP","AP50","AP75","APs","APm","APl",
                        "AR1","AR10","AR100","ARs","ARm","ARl"] if c in d.columns]
    d = d[keep]
    for c in keep:
        if c != "Class":
            d[c] = pd.to_numeric(d[c], errors="coerce")
    d["Model"] = label
    return d

df = pd.concat([
    tag_df(df_original, "Original + YOLOv8"),
    tag_df(df_zero_dce, "Zero-DCE++ + YOLOv8"),
    tag_df(df_nafnet,   "Zero-DCE++ + NAFNet + YOLOv8")
], ignore_index=True)

if "Class" not in df.columns:
    raise ValueError("Could not find a 'Class' column after normalization. Check your CSV headers.")

present = [c for c in CLASS_ORDER if c in df["Class"].unique().tolist()]
if present:
    df["Class"] = pd.Categorical(df["Class"], categories=present, ordered=True)

def plot_grouped(metric, title, fname, ylim=None):
    if metric not in df.columns:
        print(f"Skip {metric}: not in CSV"); return

    classes_present = df["Class"].dropna().unique().tolist()
    order_cls = [c for c in CLASS_ORDER if c in classes_present] or sorted(classes_present)

    pivot = df.pivot_table(index="Class", columns="Model",
                           values=metric, aggfunc="mean").loc[order_cls]

    models = list(pivot.columns)
    x = np.arange(len(pivot.index))
    w = 0.9 / max(1, len(models))  

    fig, ax = plt.subplots(figsize=(20, 14)) 
    for i, m in enumerate(models):
        ax.bar(x + (i - (len(models)-1)/2)*w, pivot[m].values, width=w,
               label=m, color=COLOR.get(m, None))
        for xi, yi in zip(x, pivot[m].values):
            if np.isfinite(yi):
                ax.text(xi + (i - (len(models)-1)/2)*w, yi, f"{yi:.3f}",
                        ha="center", va="bottom", fontsize=11, fontweight="bold") 

    ax.set_xticks(x)
    ax.set_xticklabels(pivot.index, rotation=30, ha="right")
    ax.set_title(title, pad=12)
    ax.set_xlabel("Class")
    if ylim:
        ax.set_ylim(ylim)
    ax.yaxis.set_visible(False)
    ax.set_ylabel("")
    for s in ["left", "right", "top"]:
        ax.spines[s].set_visible(False)
    ax.grid(False)

    ax.legend(title="Model")
    plt.tight_layout()
    out_path = os.path.join(out_dir, fname)
    plt.savefig(out_path, dpi=300)  
    plt.close()
    print("Saved:", out_path)
plot_grouped("AP",    "Per-class AP (mAP50–95)", "per_class_AP.png")
plot_grouped("AR100", "Per-class Recall (AR@100)", "per_class_AR100.png")


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\449320049.py:75: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result\class\per_class_AP.png


C:\Users\ibado\AppData\Local\Temp\ipykernel_18872\449320049.py:75: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(index="Class", columns="Model",


Saved: D:\1. UoB Data Science\MSc FINAL PROJECT\1 . computer vision task\my project\data\2. YOLOv8\test_result\class\per_class_AR100.png
